In [1]:
import pandas as pd
import sys
# Get the directory of the current script
sys.path.append("../")

from lib.uid import gen_uid
from lib.clean import clean_names, clean_races, clean_sexes

In [2]:
def read_personnel_and_complaints():
    dfa = pd.read_excel("../../../data/SC/2023-7-24/export_employment_officer-2023-07-24-20231220144255.xlsx")

    dfb = pd.read_csv("../../../data/SC/2023-7-24/export_officer_inf_clean.csv")

    return dfa, dfb


In [3]:
dfa, dfb = read_personnel_and_complaints()
dfa

,Academy ID,Last Name,First name,Middle,Employing Org,Employment Start Date,Employment End Date
0,4855-7813,Aaron,Cameron,Michael,Charleston Co. Sheriff's Office,2021-05-07 00:00:00,NaN
1,7382-1010,Aaron,Matthew,Charles,Beaufort Co. Sheriff's Office,2019-09-09 00:00:00,NaN
2,7382-1010,Aaron,Matthew,Charles,York Co. Sheriff's Office,2017-05-31 00:00:00,2019-09-05 00:00:00
3,7382-1010,Aaron,Matthew,Charles,York Co. Detention Center,2014-07-07 00:00:00,2017-05-31 00:00:00
4,2971-6450,Aarons,Alexis,NaN,Richland Co. Sheriff's Department,2018-07-23 00:00:00,2019-07-16 00:00:00
...,...,...,...,...,...,...,...
121415,5589-2059,Zwolak,Brian,J.,Lexington Co. Sheriff's Office,2016-12-05 00:00:00,NaN
121416,5589-2059,Zwolak,Brian,J.,SC Criminal Justice Academy,2015-06-17 00:00:00,2016-12-02 00:00:00
121417,5589-2059,Zwolak,Brian,J.,Columbia Police Dept.,2007-05-21 00:00:00,2015-06-12 00:00:00
121418,8707-0499,Zylstra,Scott,L.,Lexington Co. Sheriff's Office,2013-12-09 00:00:00,NaN


In [4]:
dfb

dfb = dfb[["race", "sex", "Academy ID" ]]

In [5]:
dfa = dfa.rename(columns={"Last Name": "last_name", "First name": "first_name", "Middle": "middle_name", "Employment Start Date": "hire_date", "Employment End Date": "left_date", "Employing Org": "agency"})

dfa = dfa.pipe(clean_names, ["first_name", "last_name", "middle_name"])

df = pd.merge(dfa, dfb, on="Academy ID")

df = df.pipe(gen_uid, ["first_name", "last_name", "middle_name", "agency"])

def clean_dates(df):
    # Convert the 'ST_DATE' column to datetime
    df['hire_date'] = pd.to_datetime(df['hire_date'], errors="coerce")
    
    # Format the dates as 'Month Day, Year'
    df['hire_date'] = df['hire_date'].dt.strftime('%m/%d/%Y')

    df['left_date'] = pd.to_datetime(df['left_date'], errors="coerce")


    df['left_date'] = df['left_date'].dt.strftime('%m/%d/%Y')
    return df

df = df.pipe(clean_dates)

df["state"] = "SC"

In [6]:
df

,Academy ID,last_name,first_name,middle_name,agency,hire_date,left_date,race,sex,uid,state
0,4855-7813,aaron,cameron,michael,Charleston Co. Sheriff's Office,05/07/2021,NaN,white,male,4a96c473759a136343b2e09403e99d41,SC
1,7382-1010,aaron,matthew,charles,Beaufort Co. Sheriff's Office,09/09/2019,NaN,white,male,1913aee3901806d0fc69fd819826a542,SC
2,7382-1010,aaron,matthew,charles,York Co. Sheriff's Office,05/31/2017,09/05/2019,white,male,9fe83dce0fec35849edc34c70b434318,SC
3,7382-1010,aaron,matthew,charles,York Co. Detention Center,07/07/2014,05/31/2017,white,male,39b98a136d8a95028107102d926e2cb4,SC
4,5220-2074,aarons,david,lee,Goose Creek Police Dept.,03/05/1998,02/21/2020,white,male,ad1d8ae7602e2d9ef34ac71baa18a15f,SC
...,...,...,...,...,...,...,...,...,...,...,...
73874,5589-2059,zwolak,brian,j,Lexington Co. Sheriff's Office,12/05/2016,NaN,white,male,d67f36ad677755fce508a392df92b301,SC
73875,5589-2059,zwolak,brian,j,SC Criminal Justice Academy,06/17/2015,12/02/2016,white,male,6b7adea94475f9df30c645475cd0cd71,SC
73876,5589-2059,zwolak,brian,j,Columbia Police Dept.,05/21/2007,06/12/2015,white,male,65897aa75f12de0b9b04f502cc4b2a81,SC
73877,8707-0499,zylstra,scott,l,Lexington Co. Sheriff's Office,12/09/2013,NaN,white,male,bb62c6d8beb742ba04bf36f54275b9f1,SC


In [7]:
df.to_csv("../../../data/SC/clean/personnel-sc-clean.csv", index=False)

In [ ]:
df_events = df[["uid", "agency", "state", "hire_date", "left_date"]]

df_events = df_events[~((df_events.left_date.fillna("") == ""))]

# Unpivot the DataFrame
df_long = df_events.melt(id_vars=['uid', 'agency', 'state'], 
                  value_vars=['hire_date', 'left_date'], 
                  var_name='event_type', 
                  value_name='event_date')

# Generate event_uid using gen_uid function
df_long = df_long.pipe(gen_uid, ['uid', 'event_type', 'event_date'], "event_uid")

events_df= df_long[df_long.duplicated(subset=["uid"], keep=False)]

# # officers that we know have moved
events_df.sort_values("uid")


events_df.to_csv("./../../../data/SC/match/events_sc_dropped_na_left_dates.csv", index=False)

In [ ]:
dup_uids_list = [x for x in events_df["uid"]]

per_df = df[["first_name", "middle_name", "last_name", "race", "sex", "uid", "state"]]

per_df = per_df[per_df.uid.isin(dup_uids_list)]

per_df.to_csv("../../../data/SC/match/personnel_sc_dropped_na_left_dates.csv", index=False)